In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import scipy.stats as stat
import statsmodels.api as sm
import scikit_posthocs as sp

pd.set_option('display.max_columns', 9999)

%matplotlib inline

## Data Cleaning

In [2]:
# finding all opioids and nsaids given to patients

opioids = ['hydromorphone','hydrocodone','morphine','oxycodone','oxymorphone','codeine','fentanyl','meperidine',
          'tramadol','carfentanil','percocet','norco']

nsaids = ['acetaminophen','amitriptyline', 'doxepin', 'imipramine', 'desipramine', 'nortriptyline',
         'ibuprofen', 'naproxen', 'diclofenac', 'piroxicam', 'sulindac', 'indomethacin', 'ketorolac', 'meloxicam', 
          'celecoxib', 'ketoprofen', 'oxaprozin', 'toradol', 'valdecoxib', 'bextra', 'rofecoxib', 'vioxx', 'gabapentin', 
          'neurontin', 'cyclobenzaprine', 'duloxetine', 'cymbalta', 'pregabalin', 'lyrica', 'venlafaxine', 'effexor', 
          'tylenol', 'voltaren', 'naprosyn', 'paracetamol', 'aspirin']


def opioids_nsaids (x, li):

    for i in li:
        if i in str(x).lower():
            return 1
        
    return 0

In [ ]:
med = pd.read_csv('eicu-collaborative-research-database-2.0/medication.csv.gz')
pts = pd.read_csv('eicu-collaborative-research-database-2.0/patient.csv.gz')
hos = pd.read_csv('eicu-collaborative-research-database-2.0/hospital.csv.gz')

In [ ]:
med = med[med.drugordercancelled == 'No']

cols_drop = ['medicationid',
             'drugorderoffset',
             'drugivadmixture',
             'drugordercancelled',
             'drughiclseqno',
             'routeadmin',
             'loadingdose',
             'prn',
             'gtc'
            ]

med.drop(columns=cols_drop, inplace=True)

med['opioid'] = med.drugname.apply(lambda x: opioids_nsaids(x, opioids))
med['nsaid'] = med.drugname.apply(lambda x: opioids_nsaids(x, nsaids))

med = med[(med.opioid == 1) | (med.nsaid == 1)]

mg = med.groupby(['patientunitstayid']).agg({'opioid': lambda x:sum(x), 'nsaid': lambda x:sum(x)})
mg[mg != 0] = 1

In [ ]:
def op_find (x, arr):
    try:
        return arr['opioid'][x]
    except:
        return np.nan
    
def nsaid_find (x, arr):
    try:
        return arr['nsaid'][x]
    except:
        return np.nan

In [ ]:
pts = pts[pts.gender != 'Unknown']
pts = pts[pts.gender != 'Other']

pts['opioid'] = pts['patientunitstayid'].apply(lambda x: op_find(x, mg))
pts['nsaid'] = pts['patientunitstayid'].apply(lambda x: nsaid_find(x, mg))

pts = pd.merge(pts, hos, on=['hospitalid'], how = 'left')

In [ ]:
# fill missing values

pts['apacheadmissiondx'].fillna(value = 'N/A', inplace = True)
pts['hospitaladmitsource'].fillna(value = 'Unknown', inplace = True)

def age (x):
    if x == '> 89':
        return 90
    else:
        return int(x)

pts['age'].fillna(value = 0, inplace = True)
pts['age'] = pts['age'].apply(lambda x: age(x))

pts.opioid.fillna(0, inplace = True)
pts.nsaid.fillna(0, inplace = True)

pts.opioid.replace(1.0,True,inplace=True)
pts.opioid.replace(0.0,False,inplace=True)

pts.nsaid.replace(1.0,True,inplace=True)
pts.nsaid.replace(0.0,False,inplace=True)

pts['painmeds'] = pts.opioid|pts.nsaid

In [ ]:
pts.head()

In [ ]:
pts.to_csv('final_patients.csv')

## Plotting
Ethnicity vs. Pain Medication Administration (Either, NSAID Only, Opioid Only)

In [ ]:
plt.figure(figsize=(14, 4.5))
sns.barplot(x='ethnicity',y='painmeds',data=pts, estimator=np.mean)
plt.xlabel('Patient Ethnicity')
plt.ylabel('Proportion Receiving Pain Medications')

In [ ]:
plt.figure(figsize=(14, 4.5))
sns.barplot(x='ethnicity',y='nsaid',data=pts, estimator=np.mean)
plt.xlabel('Patient Ethnicity')
plt.ylabel('Proportion Receiving NSAIDs')

In [ ]:
plt.figure(figsize=(14, 4.5))
sns.barplot(x='ethnicity',y='opioid',data=pts, estimator=np.mean)
plt.xlabel('Patient Ethnicity')
plt.ylabel('Proportion Receiving Opioids')

## Checking Significance
Chi-squared testing

### Test 1:
H<sub>0</sub>: Pain medication administration is independent of patient ethnicity.  
H<sub>1</sub>: Pain medication administration is not independent of patient ethnicity.

In [ ]:
ct_eth = pd.crosstab(pts.painmeds, pts.ethnicity)
ct_eth

In [ ]:
c, p, dof, expected = stat.chi2_contingency(ct_eth)

print('Pearson Chi-Square: ', c)
print('p-value: ', p)

In [ ]:
# We conclude there is a relationship between pain meds and ethnicity.

___
##### Checking between groups

In [ ]:
c_aa_ = ct_eth[['African American','Caucasian']]
c_aa_

In [ ]:
c, p, dof, expected = stat.chi2_contingency(c_aa_)

print('Pearson Chi-Square: ', c)
print('p-value: ', p)

___
### Test 2:
H<sub>0</sub>: NSAID administration is independent of patient ethnicity.  
H<sub>1</sub>: NSAID administration is not independent of patient ethnicity.

In [ ]:
ct_ns = pd.crosstab(pts.nsaid, pts.ethnicity)
ct_ns

In [ ]:
c, p, dof, expected = stat.chi2_contingency(ct_ns)

print('Pearson Chi-Square: ', c)
print('p-value: ', p)

In [ ]:
c_aa_ns = ct_ns[['African American','Caucasian']]
c_aa_ns # do for all groups

In [ ]:
c, p, dof, expected = stat.chi2_contingency(c_aa_ns)

print('Pearson Chi-Square: ', c)
print('p-value: ', p)

In [ ]:
# We conclude there is a relationship between NSAIDs and ethnicity.

##### Test 3:
H<sub>0</sub>: Opioid administration is independent of patient ethnicity.  
H<sub>1</sub>: Opioid administration is not independent of patient ethnicity.

In [ ]:
ct_op = pd.crosstab(pts.opioid, pts.ethnicity)
ct_op

In [ ]:
c, p, dof, expected = stat.chi2_contingency(ct_op)

print('Pearson Chi-Square: ', c)
print('p-value: ', p)

In [ ]:
c_aa = ct_op[['African American','Caucasian']]
c_aa # do for all groups

In [ ]:
c, p, dof, expected = stat.chi2_contingency(c_aa)

print('Pearson Chi-Square: ', c)
print('p-value: ', p)

In [ ]:
# We conclude there is a relationship between Opioids and ethnicity.

___
### Morphine Equivalence Analysis

In [ ]:
opi = pd.read_csv('opioids_final.csv')
opi = pd.merge(opi, pts, on=['patientunitstayid'], how = 'left')

In [ ]:
opi['mme_test'] = (opi.dos_mg.multiply(opi.doses_per_day)).multiply(opi.mme)

In [ ]:
opi['mme_per_day']= np.multiply(opi['dos_mg'],opi['doses_per_day'])
opi['mme_per_day']= np.multiply(opi['mme_per_day'],opi['mme'])

In [ ]:
opi.head()

In [ ]:
opi_stat = opi[['patientunitstayid','mme_per_day']]
#opi_stat.fillna(value='Other/Unknown', inplace=True)

opi_stat.head()

In [ ]:
opi.describe()

In [ ]:
opi[opi.mme_test < 0]

In [ ]:
sns.displot(opi_stat, x='mme_per_day', aspect=5, height=8)

In [ ]:
plt.figure(figsize=(14, 4.5))
sns.barplot(x='ethnicity',y='mme_per_day',data=opi_stat, estimator=np.mean)
plt.xlabel('Ethnicity')
plt.ylabel('MME per Day')

In [ ]:
# Testing for Normal Distribution

opi_stat_c = list(opi_stat[opi_stat.ethnicity == 'Caucasian'].mme_per_day)
opi_stat_aa = list(opi_stat[opi_stat.ethnicity == 'African American'].mme_per_day)
opi_stat_a = list(opi_stat[opi_stat.ethnicity == 'Asian'].mme_per_day)
opi_stat_h = list(opi_stat[opi_stat.ethnicity == 'Hispanic'].mme_per_day)
opi_stat_na = list(opi_stat[opi_stat.ethnicity == 'Native American'].mme_per_day)
opi_stat_o = list(opi_stat[opi_stat.ethnicity == 'Other/Unknown'].mme_per_day)

k_c, p_c = stat.kstest(opi_stat_c, 'norm')
k_aa, p_aa = stat.kstest(opi_stat_aa, 'norm')
k_a, p_a = stat.kstest(opi_stat_a, 'norm')
k_h, p_h = stat.kstest(opi_stat_h, 'norm')
k_na, p_na = stat.kstest(opi_stat_na, 'norm')
k_o, p_o = stat.kstest(opi_stat_o, 'norm')

print('Caucasian: k-value:', k_c, 'p-value:', p_c)
print('African America: k-value:', k_aa, 'p-value:', p_aa)
print('Asian: k-value:', k_a, 'p-value:', p_a)
print('Hispanic: k-value:', k_h, 'p-value:', p_h)
print('Native American: k-value:', k_na, 'p-value:', p_na)
print('Other: k-value:', k_o, 'p-value:', p_o)

In [ ]:
# Non-normal distributions, using non-parametric Kruskal-Wallis H-test

stat.kruskal(opi_stat_c,opi_stat_aa,opi_stat_a,opi_stat_h,opi_stat_na,opi_stat_o)

In [ ]:
from statsmodels.stats.multicomp import pairwise_tukeyhsd

# Multiple pairwise comparison (Tukey HSD) - whats the nonparametric equivalent?

m_comp = pairwise_tukeyhsd(endog=opi_stat['mme_per_day'], groups=opi_stat['ethnicity'], alpha=0.05)
m_comp.summary()

In [ ]:
nemenyi = sp.posthoc_nemenyi(opi_stat, 'mme_per_day', 'ethnicity')
nemenyi

In [ ]:
# Bonferroni correction

pvals = [] # grab pvals from nemenyi df above by columns

for i in nemenyi.columns:
    pvals = pvals + list(nemenyi[i])

from statsmodels.stats.multitest import multipletests

p_adjusted = multipletests(pvals, alpha=0.05, method='bonferroni')

In [ ]:
print('Corrected alpha for Bonferroni method: ', p_adjusted[3])

In [ ]:
group1 = []
group2 = []

for col in nemenyi.columns:
    for row in nemenyi.index:
        group1.append(col)
        group2.append(row)

In [ ]:
d = {'group1': group1, 'group2': group2, 'p_vals': p_adjusted[1], 'reject': p_adjusted[0]}

nemenyi_comp = pd.DataFrame(data=d)

In [ ]:
nemenyi_comp

In [ ]:
# Still need to remove duplicates 
# (i.e., group1 = African American, group2 = Caucasian &  group1 = Caucasian, group2 = African American)

### Chi Square for Specific Groups

In [ ]:
opi_eth = pd.get_dummies(pts, columns=['ethnicity'])

In [ ]:
opi_eth.head()

In [ ]:
african_american = pd.crosstab(opi_eth.opioid, opi_eth['ethnicity_African American'])
african_american

In [ ]:
c, p, dof, expected = stat.chi2_contingency(african_american)

print('Pearson Chi-Square: ', c)
print('p-value: ', p)

In [ ]:
asian = pd.crosstab(opi_eth.opioid, opi_eth['ethnicity_Asian'])
asian

In [ ]:
c, p, dof, expected = stat.chi2_contingency(asian)

print('Pearson Chi-Square: ', c)
print('p-value: ', p)

In [ ]:
caucasian = pd.crosstab(opi_eth.opioid, opi_eth['ethnicity_Caucasian'])
caucasian

In [ ]:
c, p, dof, expected = stat.chi2_contingency(caucasian)

print('Pearson Chi-Square: ', c)
print('p-value: ', p)

In [ ]:
hispanic = pd.crosstab(opi_eth.opioid, opi_eth['ethnicity_Hispanic'])
hispanic

In [ ]:
c, p, dof, expected = stat.chi2_contingency(hispanic)

print('Pearson Chi-Square: ', c)
print('p-value: ', p)

In [ ]:
namerican = pd.crosstab(opi_eth.opioid, opi_eth['ethnicity_Native American'])
namerican

In [ ]:
c, p, dof, expected = stat.chi2_contingency(namerican)

print('Pearson Chi-Square: ', c)
print('p-value: ', p)

In [ ]:
other = pd.crosstab(opi_eth.opioid, opi_eth['ethnicity_Other/Unknown'])
other

In [ ]:
c, p, dof, expected = stat.chi2_contingency(other)

print('Pearson Chi-Square: ', c)
print('p-value: ', p)

___
### Feature Engineering

In [ ]:
#apache score

apache = pd.read_csv('eicu-collaborative-research-database-2.0/apachePatientResult.csv.gz')

apache = apache[apache['apacheversion']=='IV']
apache_scores = apache[['patientunitstayid','apachescore','actualventdays']]
pts = pd.merge(pts, apache_scores, on=['patientunitstayid'], how = 'right')

In [ ]:
pts.actualventdays.fillna(value=0, inplace=True)

In [ ]:
# gcs score

gcs = pd.read_csv('eicu-collaborative-research-database-2.0/apachePredVar.csv.gz')

gcs = gcs[['patientunitstayid','verbal', 'motor', 'eyes']]
gcs['gcs_score'] = gcs['verbal'] + gcs['motor'] + gcs['eyes']
gcs = gcs[['patientunitstayid', 'gcs_score']]

pts = pd.merge(pts, gcs, on=['patientunitstayid'], how = 'right')

In [ ]:
pts['from_OR'] = pts['unitadmitsource'] == 'Operating Room'

In [ ]:
# admission HR

systolic = pd.read_csv('eicu-collaborative-research-database-2.0/vitalPeriodic.csv.gz')
systolic = systolic[['patientunitstayid', 'observationoffset', 'heartrate']]
systolic.to_csv('heartrate.csv')

In [ ]:
grouped_systolic = systolic.groupby('patientunitstayid').apply(lambda x: x.sort_values('observationoffset'))

first_values = grouped_systolic.drop_duplicates(subset='patientunitstayid', keep='first')
first_values['hr_over100'] = first_values['heartrate'] >= 100
first_values = first_values[['heartrate', 'hr_over100']]

first_values.to_csv('heartrate2.csv')

In [ ]:
first_values = pd.read_csv('heartrate2.csv')
pts = pd.merge(pts, first_values, on=['patientunitstayid'], how = 'right')

In [ ]:
pts['ICU_duration'] = (pts['unitdischargeoffset'] - pts['hospitaladmitoffset'])/1440

In [ ]:
# ordered protocols

cpg = pd.read_csv('eicu-collaborative-research-database-2.0/carePlanGeneral.csv.gz')

cpg = cpg[cpg.cplgroup == 'Ordered Protocols']
cpg.drop(columns=['cplgeneralid', 'activeupondischarge', 'cplitemoffset', 'cplgroup'], inplace = True)
cpg.drop_duplicates(inplace=True)

cpg = cpg.groupby('patientunitstayid')['cplitemvalue'].apply(list).reset_index(name='orderedprotocols')
cpg.head()

In [ ]:
from sklearn.preprocessing import MultiLabelBinarizer
# one hot ecoding from list
mlb = MultiLabelBinarizer()
cpg = cpg.join(pd.DataFrame(mlb.fit_transform(cpg.pop('orderedprotocols')), columns=mlb.classes_, index=cpg.index))

In [ ]:
cpg.head()

In [ ]:
pts.head()

In [ ]:
pts = pts[(pts.admissionheight > 100)]
pts = pts[(pts.admissionweight > 0) & (pts.admissionweight < 200)]

In [ ]:
#pts.dropna(subset=['admissionheight','admissionweight'], inplace=True)
pts.drop(columns=['Unnamed: 1'], inplace=True)

In [ ]:
pts['admissionheight'] = pts['admissionheight'].replace(0, pts['admissionheight'].median())
pts['admissionweight'] = pts['admissionweight'].replace(0, pts['admissionweight'].median())
pts['age'] = pts['age'].replace(0, pts['age'].median())

In [ ]:
pts['BMI'] = (pts.admissionweight/pts.admissionheight/pts.admissionheight)*10000

In [ ]:
pts.head()

In [ ]:
# pain scores
pain = pd.read_csv('pain.csv')
pts = pd.merge(pts, pain, on=['patientunitstayid'], how = 'inner')

In [ ]:
pts.info()

In [ ]:
pts_final = pts[['patientunitstayid',
                 'gender', 
                 'age', 
                 'ethnicity',
                 'BMI', 
                 'actualventdays',
                 'ICU_duration',
                 'gcs_score', 
                 'apachescore', 
                 'from_OR',
                 'heartrate',
                 'teachingstatus',
                 'numbedscategory',
                 'opioid', 
                 'nsaid',
                 'painmeds',
                 'initialPain',
                 'finalPain']]

In [ ]:
pts_final.head()

In [ ]:
pts_final.to_csv('final_patients.csv')

In [ ]:
pts_final = pd.merge(pts_final, opi_stat, on=['patientunitstayid'], how = 'left')
pts_final.head()

In [ ]:
pts_final.mme_per_day.fillna(0.0,inplace=True)

In [ ]:
pts_final.dropna(inplace=True)

In [ ]:
pts_final["painmeds"] = pts_final["painmeds"].astype(int)
pts_final["opioid"] = pts_final["opioid"].astype(int)
pts_final["nsaid"] = pts_final["nsaid"].astype(int)
pts_final["from_OR"] = pts_final["from_OR"].astype(int)
#pts_final["hr_over100"] = pts_final["hr_over100"].astype(int)

In [ ]:
pts_final.teachingstatus.replace('t', 1, inplace=True)
pts_final.teachingstatus.replace('f', 0, inplace=True)

pts_final.gender.replace('Male', 1, inplace=True)
pts_final.gender.replace('Female', 0, inplace=True)

In [ ]:
pts_final.head()

In [ ]:
pts_final.to_csv('pts_no_dummies.csv')

In [ ]:
pts_final = pd.read_csv('pts_no_dummies.csv')

In [ ]:
pts_final = pd.get_dummies(pts_final, columns = ['numbedscategory'], drop_first = False)
pts_final = pd.get_dummies(pts_final, columns = ['ethnicity'], drop_first = False)

In [ ]:
pts_final = pd.merge(pts_final, cpg, on=['patientunitstayid'])

In [ ]:
pts_final.head()

In [ ]:
pts_final.to_csv('pts_dummies.csv')

In [ ]:
pts_final.info()

In [ ]:
pts_drop = pts_final.dropna()

In [ ]:
pts_drop.info()

In [ ]:
pts_drop.head()

In [ ]:
pts_drop.to_csv('pts_dummies.csv')

___
### Multivariate Analysis

In [ ]:
def logreg_df(df, norm_cols, drop_cols, y_col, drop=False, reg=0):
    """
    df (string): CSV path.
    cat_cols (list): categorical columns to one-hot-encode.
    norm_cols (list): columns in df to normalize.
    drop_cols (list): columns to drop.
    y_col (string): variable of interest.
    """
    
    #df = pd.read_csv('df')
    #df = pd.get_dummies(df, columns = cat_cols, drop_first = drop)
    
    from sklearn.preprocessing import MinMaxScaler
    from sklearn.model_selection import train_test_split
    
    from sklearn.linear_model import LogisticRegression
    from sklearn.datasets import load_iris
    
    df[norm_cols] = MinMaxScaler().fit_transform(df[norm_cols])
    
    if reg == 0:
        X = df.drop(columns = drop_cols)
        y = df[y_col]

        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)
        
        log = LogisticRegression(penalty='l1',solver='liblinear')
        res = log.fit(X, y)
        
        #print(res)
        #print(res.coef_, res.intercept_)
        
        logit_model = sm.Logit(y, X)
        result = logit_model.fit(maxiter=1)

        print(result.summary2())
        
        params = result.params
        conf = result.conf_int()
        conf['Odds Ratio'] = params
        conf.columns = ['5%', '95%', 'Odds Ratio']
        print(np.exp(conf))
        print('==========================')
        
    elif reg == 1:
        X = df.drop(columns = drop_cols)
        y = df[y_col]
        X = sm.add_constant(X)
        
        ols_model = sm.OLS(y,X)
        result = ols_model.fit()
        
        print(result.summary())
        
        y_pred = result.predict(X)
    
    return result, y_pred, y, X

#### Painmeds by Ethnicity

In [ ]:
eth = ['ethnicity_Caucasian', 'ethnicity_African American', 'ethnicity_Asian',
      'ethnicity_Hispanic', 'ethnicity_Native American', 'ethnicity_Other/Unknown']

for i in eth:
    norm = ['actualventdays','age', 'gcs_score', 'apachescore']
    dp = ['opioid', 'nsaid', 'patientunitstayid','painmeds','mme_per_day',i]
    
    print("Compared to ", i)
    logreg_df(pts_final, norm, dp, 'painmeds')

In [ ]:
# In the above Caucasian baseline, there was no significance with African Americans, so why is it not both ways?

In [ ]:
# basically the above is saying compared to caucasians, hispanic and native american are signiciant 
# - both are more likely to receive pain meds

#### Ethnicity vs. Receiving Opioids

In [ ]:
eth = ['ethnicity_Caucasian', 'ethnicity_African American', 'ethnicity_Asian',
      'ethnicity_Hispanic', 'ethnicity_Native American', 'ethnicity_Other/Unknown']

for i in eth:
    norm = ['actualventdays','age','gcs_score', 'apachescore']
    dp = ['patientunitstayid','painmeds','opioid', i]
    
    print("Compared to ", i)
    logreg_df(pts_final, norm, dp, 'opioid', reg=0)

#### Ethnicity vs. MME

In [ ]:
eth = ['ethnicity_Caucasian', 'ethnicity_African American', 'ethnicity_Asian',
      'ethnicity_Hispanic', 'ethnicity_Native American', 'ethnicity_Other/Unknown']

models = []

for i in eth:
    norm = ['actualventdays','age', 'gcs_score', 'apachescore']
    dp = ['mme_per_day','patientunitstayid','painmeds','opioid', i]
    
    print("Compared to ", i)
    ols, y_pred, y_true, X = logreg_df(pts_final, norm, dp, 'mme_per_day', reg=1)
    models.append((y_pred, y_true, X))

In [ ]:
plt.figure(figsize=(14, 4.5))
plt.plot(models[0][0], label="OLS", alpha=0.5)
plt.plot(models[0][1], label="True", alpha=0.5)
plt.legend(loc='best')
plt.xlabel('Patient Number')
plt.ylabel('MME per Day')

#### Figures

In [ ]:
pts_figs = pd.read_csv('pts_no_dummies.csv')
pts_figs.head()

In [ ]:
plt.figure(figsize=(14, 4.5))
sns.barplot(x='ethnicity',y='painmeds',data=pts_figs, estimator=np.mean)
plt.xlabel('Patient Ethnicity')
plt.ylabel('Proportion Receiving Pain Medications')

In [ ]:
plt.figure(figsize=(14, 4.5))
sns.barplot(x='ethnicity',y='nsaid',data=pts_figs, estimator=np.mean)
plt.xlabel('Patient Ethnicity')
plt.ylabel('Proportion Receiving NSAIDs')

In [ ]:
plt.figure(figsize=(14, 4.5))
sns.barplot(x='ethnicity',y='opioid',data=pts_figs, estimator=np.mean)
plt.xlabel('Patient Ethnicity')
plt.ylabel('Proportion Receiving Opioids')

In [ ]:
plt.figure(figsize=(14, 4.5))
sns.barplot(x='ethnicity',y='mme_per_day',data=pts_figs, estimator=np.mean)
plt.xlabel('Patient Ethnicity')
plt.ylabel('MME per Day')

___
### Other Models

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score,mean_squared_error

X = pts_final.drop(columns = ['mme_per_day','patientunitstayid','painmeds','opioid'])
y = pts_final['mme_per_day']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)

regressor = RandomForestRegressor(n_estimators = 100, random_state = 0)
regressor.fit(X_train, y_train)

In [ ]:
y_pred = regressor.predict(X_test)

mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
rmse

In [ ]:
# fake patient with figure

In [ ]:
# https://towardsdatascience.com/multiple-imputation-with-random-forests-in-python-dec83c0ac55b